In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Caminho para o arquivo JSON da conta de serviço
key_path = r"C:\BIGQUERY_KEY_PATH\rock-tower-454219-t2-f2eeba73d76a.json"
# Definir o escopo de acesso
scopes = ['https://www.googleapis.com/auth/cloud-platform']


# Carregar as credenciais
credentials = service_account.Credentials.from_service_account_file(key_path)

# Criar o cliente BigQuery
client = bigquery.Client(project='rock-tower-454219-t2', credentials=credentials)

In [ ]:
job = client.get_job('bquxjob_508ab77d_1961c25c639d')  # Substitua pelo ID do seu job
results = job.to_dataframe()